## VA model (with feedback!)

In [1]:
from EMO_AI.all import *

In [2]:
"need to change the name, since there's a get_model() function in torch"
model = get_model("C:/Users/Kelvin/Desktop/ai_sheng/emo_0.pt",inference_only=False)
model.classifier

c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\models\auto\modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Sequential(
  (0): Dropout(p=0.05, inplace=False)
  (1): Linear(in_features=768, out_features=768, bias=True)
  (2): Mish()
  (3): Dropout(p=0.05, inplace=False)
  (4): Linear(in_features=768, out_features=6, bias=True)
)

In [4]:
get_output("Hatred", model)

'anger'

In [5]:
from pathlib import Path as P
import os

In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [19]:
import gc
def report_gpu():
    gc.collect()
    torch.cuda.empty_cache()

In [20]:
report_gpu()

## We need to train in batch, which should be easy

### check out the RoBERTa example

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
class EmoDataset(Dataset):
    def __init__(self, path):
        super().__init__()
        self.data_column = "text"
        #self.class_column = "class"
        self.class_column = "label(int)"
        self.data = pd.read_csv(path, sep=";", header=None, names=[self.data_column, self.class_column],
                               engine="python")

    def __getitem__(self, idx):
        return self.data.loc[idx, self.data_column], label2int[self.data.loc[idx, self.class_column]]

    def __len__(self):
        return self.data.shape[0]

class TokenizersCollateFn:
    def __init__(self, max_tokens=512):

        """
        ## RoBERTa uses BPE tokenizer similar to GPT
        t = ByteLevelBPETokenizer(
            "tokenizer/vocab.json",
            "tokenizer/merges.txt"
        )
        t._tokenizer.post_processor = BertProcessing(
            ("</s>", t.token_to_id("</s>")),
            ("<s>", t.token_to_id("<s>")),
        )
        t.enable_truncation(max_tokens)
        t.enable_padding(length=max_tokens, pad_id=t.token_to_id("<pad>"))
        self.tokenizer = t
        """
        self.tokenizer = get_tokenizer(max_tokens)

    def __call__(self, batch):
        encoded = self.tokenizer.encode_batch([x[0] for x in batch])
        sequences_padded = torch.tensor([enc.ids for enc in encoded])
        attention_masks_padded = torch.tensor([enc.attention_mask for enc in encoded])
        labels = torch.tensor([x[1] for x in batch])
        
        return (sequences_padded, attention_masks_padded), labels

In [ ]:
"We need train-test split!"
"read from dataset -> split it -> save them separately"
from pathlib import Path as P
import pandas as pd

...

In [ ]:
shuffle = True
batch_size = 64
DataLoader(EmoDataset(P("plurk_dataset_manual.csv")), batch_size, shuffle, collate_fn=TokenizerCollateFN())

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [ ]:
model.to(device)

EmoModel(
  (base_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,

In [22]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    i = 0
    for data in data_list:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model((inputs[0].to(device), inputs[1].to(device)))
        loss = criterion(outputs, torch.tensor([label2int[labels]]).to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
            report_gpu()
        
        i += 1

print('Finished Training')

Finished Training


In [23]:
running_loss

0.08276451620622538

In [34]:
tmp = convert_text_to_tensor("生氣")
test_out = model((tmp[0].to(device), tmp[1].to(device))).argmax(1)
test_out

tensor([3], device='cuda:0')